In [1]:
import os
rutaBase = os.getcwd().replace('\\', '/') + '/'

In [2]:
os.chdir(rutaBase + '../src/')
from funciones_MODIS import *

In [3]:
# rutas de entrada y salida
rutaMODIS = 'F:/OneDrive - Universidad de Cantabria/Cartografia/MODIS/'
# rutaExport = 'F:/Proyectos/GESDIVAH/SDM/Data/PdE/MODIS/'
rutaExport = rutaBase + '../output/ET/'

In [4]:
# parámetros de la extracción de datos de MODIS
product = 'MOD13A1'
var = '500m 16 days NDVI'
# atributes = [2400, 2400, -1111950.519667, 5559752.598333, 0., 4447802.078667]
tiles = ['h17v04', 'h17v05']
dateslim=['2005-01-01', '2005-06-01']
clip = rutaBase + '../data/dem.asc'
coordsOut = 'epsg:25830'
filename = 'MODIS_' + var + '.nc'

In [5]:
MODIS_extract(rutaMODIS, product, var, tiles, dateslim=dateslim, clip=clip,
              coordsClip=coordsOut)

Generar atributos globales
dimensión:		(2400, 4800)
esquina inf. izqda.:	(-1111950.52, 3335851.56)
esquina sup. dcha.:	(      0.00, 5559752.60)

Crear máscaras
dimensión:		(  47,   49)

Importar datos
Hoja  1 de  2: h17v04
Archivo  10 de  10: MOD13A1.A2005145.h17v04.006.2015157052155.hdf
Hoja  2 de  2: h17v05
Archivo  10 de  10: MOD13A1.A2005145.h17v05.006.2015157052357.hdf


## Reordenar función

In [97]:
def MODIS_extract2(path, product, var, tiles, factor=None, dateslim=None,
                  clip=None, coordsClip='epsg:25830', verbose=True):
    """Extrae los datos de MODIS para un producto, variable y fechas dadas, transforma las coordenadas y recorta a la zona de estudio.
    
    Entradas:
    ---------
    path:       string. Ruta donde se encuentran los datos de MODIS (ha de haber una subcarpeta para cada producto)
    product:    string. Nombre del producto MODIS, p.ej. MOD16A2
    var:        string. Variable de interés dentro de los archivos 'hdf'
    factor:     float. Factor con el que multiplicar los datos para obtener su valore real (comprobar en la página de MODIS para el producto y variable de interés)
    tiles:      list. Hojas del producto MODIS a tratar
    dateslim:   list. Fechas de inicio y fin del periodo de estudio en formato YYYY-MM-DD. Si es 'None', se extraen los datos para todas las fechas disponibles
    clip:       string. Ruta y nombre del archivo ASCII que se utilizará como máscara para recortar los datos. Si es 'None', se extraen todos los datos
    coordsClip: string. Código EPSG o Proj del sistema de coordenadas al que se quieren transformar los datos. Si en 'None', se mantiene el sistema de coordenadas sinusoidal de MODIS
    verbose:    boolean. Si se quiere mostrar en pantalla el desarrollo de la función
    
    Salidas:
    --------
    Como métodos:
        data:    array (Y, X) o (dates, Y, X). Mapas de la variable de interés. 3D si hay más de un archivo (más de una fecha)
        Xcoords: array (2D). Coordenadas X de cada celda de los mapas de 'data'
        Ycoords: array (2D). Coordenadas Y de cada celda de los mapas de 'data'
        dates:   list. Fechas a las que corresponde cada uno de los maapas de 'data'
    """    
    

    os.chdir(path + product + '/')
    
    # SELECCIÓN DE ARCHIVOS
    # ---------------------
    if dateslim is not None:
        # convertir fechas límite en datetime.date
        start = datetime.strptime(dateslim[0], '%Y-%m-%d').date()
        end = datetime.strptime(dateslim[1], '%Y-%m-%d').date()
    
    dates, files = {tile: [] for tile in tiles}, {tile: [] for tile in tiles}
    for tile in tiles:
        # seleccionar archivos del producto para las hojas y fechas indicadas
        for file in [f for f in os.listdir() if (product in f) & (tile in f)]:
            year = file.split('.')[1][1:5]
            doy = file.split('.')[1][5:]
            date = datetime.strptime(' '.join([year, doy]), '%Y %j').date()
            if dateslim is not None:
                if (date>= start) & (date <= end):
                    dates[tile].append(date)
                    files[tile].append(file)
            else:
                dates[tile].append(date)
                files[tile].append(file)
    # comprobar que el número de archivos es igual en todas las hojas
    if len(set([len(dates[tile]) for tile in tiles])) > 1:
        print('¡ERROR! Diferente número de fechas en las diferentes hojas')
        MODIS_extract.files = files
        MODIS_extract.dates = dates
#         return 
    else:
        dates = np.sort(np.unique(np.array([date for tile in tiles for date in dates[tile]])))

    # ATRIBUTOS MODIS
    # ---------------
    if verbose == True: print('Generar atributos globales')
    # extraer atributos para cada hoja
    attributes = pd.DataFrame(index=tiles, columns=['ncols', 'nrows', 'Xo', 'Yf', 'Xf', 'Yo'])
    for tile in tiles:
        attributes.loc[tile,:] = hdfAttrs(files[tile][0])

    # extensión total
    Xo = np.min(attributes.Xo)
    Yf = np.max(attributes.Yf)
    Xf = np.max(attributes.Xf)
    Yo = np.min(attributes.Yo)
    # nº total de columnas y filas
    colsize = np.mean((attributes.Xf - attributes.Xo) / attributes.ncols)
    ncols = int(round((Xf - Xo) / colsize, 0))
    rowsize = np.mean((attributes.Yf - attributes.Yo) / attributes.nrows)
    nrows = int(round((Yf - Yo) / rowsize, 0))
    if verbose == True:
        print('dimensión:\t\t({0:}, {1:})'.format(ncols, nrows))
        print('esquina inf. izqda.:\t({0:>10.2f}, {1:>10.2f})'.format(Xo, Yo))
        print('esquina sup. dcha.:\t({0:>10.2f}, {1:>10.2f})'.format(Xf, Yf), end='\n\n')

    # coordenadas x de las celdas
    Xmodis = np.linspace(Xo, Xf, ncols)
    # coordenadas y de las celdas
    Ymodis = np.linspace(Yf, Yo, nrows)
    # matrices 2D con las coordenadas X e Y de cada celda
    XXmodis, YYmodis = np.meshgrid(Xmodis, Ymodis)

    if coordsClip is not None:
        # definir sistemas de referencia de coordenadas
        ProjOut = Proj(init=coordsClip)
        # https://spatialreference.org/ref/sr-org/modis-sinusoidal/
        SINUSOIDAL = Proj(projparams='PROJCS["Sinusoidal",GEOGCS["GCS_Undefined",DATUM["D_Undefined",SPHEROID["User_Defined_Spheroid",6371007.181,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.017453292519943295]],PROJECTION["Sinusoidal"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]')

        # transformar coordenadas
        Xmodis, Ymodis = transform(SINUSOIDAL, ProjOut, XXmodis.flatten(), YYmodis.flatten())
        XXmodis, YYmodis = Xmodis.reshape((nrows, ncols)), Ymodis.reshape((nrows, ncols))

    # CREAR MÁSCARAS
    # --------------
    if clip is not None:
        if verbose == True: print('Crear máscaras')
        # cargar ascii
        clipdf = ascii2df(clip)
        # extensión del ascii
        Xbo, Xbf = clipdf.columns.min(), clipdf.columns.max()
        Ybo, Ybf = clipdf.index.min(), clipdf.index.max()

        # mapa auxiliar del tamaño de los hdf
        aux = np.ones((nrows, ncols))
        # convertir en NaN celdas fuera del rectángulo de extensión de la cuenca
        maskExtent = (XXmodis >= Xbo) & (XXmodis <= Xbf) & (YYmodis >= Ybo) & (YYmodis <= Ybf)
        aux[~maskExtent] = np.nan
        # filas (maskR) y columnas (masC) en la extensión de la cuenca
        maskRows = ~np.all(np.isnan(aux), axis=1)
        maskCols = ~np.all(np.isnan(aux), axis=0)

        # recortar aux
        aux = aux[maskRows, :][:, maskCols]
        # extraer coordenadas en el rectángulo de extensión de la cuenca
        XXb, YYb = XXmodis[maskRows, :][:, maskCols], YYmodis[maskRows, :][:, maskCols]

        # convertir en NaN celdas fuera de la cuenca
        for c, (y, x) in enumerate(zip(YYb.flatten(), XXb.flatten())):
            ibasin, jbasin = np.argmin(abs(y - clipdf.index)), np.argmin(abs(x - clipdf.columns))
            if np.isnan(clipdf.iloc[ibasin, jbasin]):
                imodis, jmodis = int(c / XXb.shape[1]), c % XXb.shape[1]
                aux[imodis, jmodis] = np.nan
                maskClip = np.isnan(aux)
        if verbose == True:
            print('dimensión:\t\t({0:>4}, {1:>4})'.format(aux.shape[0], aux.shape[1]), end='\n\n')

    # IMPORTAR DATOS
    # --------------
    if verbose ==True: print('Importar datos')
        
    for d, date in enumerate(dates):
        dateStr = str(date.year) + str(date.timetuple().tm_yday).zfill(3)

        for t, tile in enumerate(tiles):
            print('Fecha {0:>2} de {1:>2}: {2}\t||\tTile {3:>2} de {4:>2}: {5}'.format(d + 1, len(dates), date,
                                                                                       t + 1, len(tiles), tile), end='\r')
            
            # localización de la hoja dentro del total de hojas
            nc, nr, xo, yf, xf, yo = attributes.loc[tile, :]
            i = int(round((Yf - yf) / (rowsize * attributes.nrows[t]), 0))
            j = int(round((Xf - xf) / (colsize * attributes.ncols[t]), 0))

            # archivo de la fecha y hoja dada
            file = [f for f in files[tile] if dateStr in f][0]
            # cargar archivo 'hdf'
            hdf = Dataset(file, format='hdf4')
            # extraer datos de la variable
            tmp = hdf[var][:]
            tmp[tmp.mask] = np.nan
            hdf.close()
            # guardar datos en un array global de la fecha
            if t == 0:
                dataD = tmp.copy()
            else:
                if (i == 1) & (j == 0):
                    dataD = np.concatenate((dataD, tmp), axis=0)
                elif (i == 0) & (j == 1):
                    dataD = np.concatenate((dataD, tmp), axis=1)
            del tmp
        
        # recortar array de la fecha con la máscara
        if clip is not None:
            dataD = dataD[maskRows, :][:, maskCols]
            dataD[maskClip] = np.nan
            
        # guardar datos en un array total
        if d == 0:
            data = dataD.copy()
        else:
            data = np.dstack((data, dataD))
        del dataD
    
    # multiplicar por el factor de escala (si existe)
    if factor is not None:
        data *= factor
        
    # reordenar el array (tiempo, Y, X)
    if len(data.shape) == 3:
        tmp = np.ones((data.shape[2], data.shape[0], data.shape[1])) * np.nan
        for t in range(data.shape[2]):
            tmp[t,:,:] = data[:,:,t]
        data = tmp.copy()
        del tmp

    # GUARDAR RESULTADOS
    # ------------------
    MODIS_extract.data = data
    MODIS_extract.dates = dates
    if clip is not None:
        MODIS_extract.Xcoords = XXb
        MODIS_extract.Ycoords = YYb
    else:
        MODIS_extract.Xcoords = XXmodis
        MODIS_extract.Ycoords = YYmodis

In [96]:
MODIS_extract.data.shape

(10, 47, 49)